In [67]:
# External imports
import torch

import torch.nn.functional as F
from torch.utils.data import IterableDataset, DataLoader

import datasets
import transformers
from transformers import AutoModel
from datasets import load_dataset
from accelerate import Accelerator

from argparse import Namespace
import logging
import json

In [52]:
# Domestic imports
from model.tokenize_input import input_string_to_tokenize_expression
from model.tokens import TOKEN_TYPE_ANSWERS, TOKEN_TYPE_EXPRESSIONS
from model.equation_interpreter import Equation
from model.vocabulary import Vocabulary

In [53]:
# Create a combined vocabulary
vocabulary = Vocabulary.construct_from_list(TOKEN_TYPE_EXPRESSIONS + TOKEN_TYPE_ANSWERS)
vectorized_sample = vocabulary.vectorize(["#", "/", "0", "-1", "[SEP]", "TT_INTEGER"])
vectorized_sample, [vocabulary.getToken(idx) for idx in vectorized_sample]

# Global variables
model_name = "JustSumAI"
repo_name = f"{model_name}_cleaned_gpt2_data"

# Setup

In [58]:
with open("./data/metadata.txt", "r") as f:
    max_length = json.loads(f.read())["max_length"]
f.close()

In [72]:
config = {
    "train_batch_size": 8,
    "valid_batch_size": 8,
    "weight_decay": 0.1,
    "shuffle_buffer": 1000,
    "learning_rate": 2e-4,
    "lr_scheduler_type": "cosine",
    "num_warmup_steps": 750,
    "gradient_accumulation_steps": 16,
    "max_train_steps": 50_000,
    "max_eval_steps": -1,
    "seq_length": max_length,
    "seed": 628,
    "save_checkpoint_steps": 50_000
}
args = Namespace(**config)

### Load dataset

In [73]:
BATCH_SIZE = args.train_batch_size
BATCH_SIZE

8

In [74]:
dataset = load_dataset(f"Dragonoverlord3000/{repo_name}")
dataset

Found cached dataset parquet (C:/Users/Hugo/.cache/huggingface/datasets/Dragonoverlord3000___parquet/Dragonoverlord3000--JustSumAI_cleaned_gpt2_data-7bb197e8e347c6a0/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 336
    })
})

In [76]:
print(json.loads(dataset["train"][0]["text"]))

[64, 9, 39, 25, 22, 21, 0, 4, 26, 23, 67, 51, 49, 49, 49, 61, 48, 59, 49, 61, 51, 49, 49, 61, 59, 48, 58, 49, 61, 58, 49, 48, 60, 49, 61, 58, 49, 44, 60, 49, 61, 58, 49, 49, 53, 60, 44, 60, 49, 61, 59, 49, 49, 49, 49, 61, 49, 53, 49, 61, 58, 61, 53, 60, 44, 60, 49, 61, 58, 49, 44, 60, 49, 49, 49, 61, 49, 53, 49, 61, 58, 53, 60, 61, 59, 49, 49, 57, 60, 49, 61, 59, 49, 49, 57, 60, 49, 61, 58, 49, 49, 57, 60, 49, 61, 59, 49, 49, 49, 53, 59, 49, 61, 57, 60, 49, 61, 59, 49, 49, 53, 60, 49, 49, 53, 59, 49, 61, 57, 60, 49, 61, 59, 49, 49, 49, 53, 58, 49, 61, 57, 60, 49, 61, 59, 49, 49, 53, 60, 49, 49, 53, 58, 49, 61, 57, 60, 49, 61, 58, 60, 59, 65]


In [77]:
train_dataloader = DataLoader(dataset["train"], shuffle=True, batch_size=BATCH_SIZE)
train_dataloader

### Load model

In [45]:
model = AutoModel.from_pretrained(f"Dragonoverlord3000/{model_name}")
model

Some weights of the model checkpoint at Dragonoverlord3000/JustSumAI were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


GPT2Model(
  (wte): Embedding(68, 1600)
  (wpe): Embedding(1024, 1600)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-47): 48 x GPT2Block(
      (ln_1): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((1600,), eps=1e-05, elementwise_affine=True)
)

In [46]:
f"GPT-2 Number of parameters: {model.num_parameters()/1_000_000:.2f}M"

'GPT-2 Number of parameters: 1477.31M'

### Define weight decay parameters

In [70]:
def get_grouped_params(model, no_decay=["bias", "LayerNorm.weight"]):
    params_with_wd, params_without_wd = [], []
    for n,p in model.named_parameters():
        if any(nd in n for nd in no_decay):
            params_without_wd.append(p)
        else:
            params_with_wd.append(p)
            
        return [{"params": params_with_wd, "weight_decay": args.weight_decay},
               {"params": params_without_wd, "weight_decay": 0.0}]

### Setting up the accelerator

In [63]:
accelerator = Accelerator()

### Setting up logging for the training loop

In [69]:
def setup_logging(project_name="JustSumAI"):
    logger = logging.getLogger(__name__)
    logging.basicConfig(
        format="%(asctime)s - %(levelname) - %(name)s - %(message)s",
        datefmt="%m/%d/&Y %H:%M:%S", level=logging.INFO, handlers=[
            logging.FileHandler(f"log/debug_{accelerator.process_index}.log"),
            logging.StreamHandler()
        ]
    )
    
    # Make sure only main thread activates logging
    if accelerator.is_main_process:
        logger.setLevel(logging.INFO)
        datasets.utils.logging.set_verbosity_debug()
        transformers.utils.logging.set_verbosity_info()
    else:
        logger.setLevel(logging.ERROR)
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()
        
    return logger

In [ ]:
def evaluate():
    model.eval()
    losses = []
    for step,batch in enumerate(eval_)

# Training Loop